# CartoCamp: Data Science + Maps workshop

Jan 27, 2017

* Andy Eschbacher, map scientist, CARTO, [@MrEPhysics](https://twitter.com/MrEPhysics), andy@carto.com
* Danny Sheehan, solutions engineer, CARTO, [@nygeog](https://twitter.com/nygeog), danny@carto.com

